In [1]:
%pip install tensorflow-gpu

     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 450kB 52.1MB/s 
     |████████████████████████████████| 3.8MB 36.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import pandas as pd

In [0]:
train_url,test_url='https://storage.googleapis.com/tf-datasets/titanic/train.csv','https://storage.googleapis.com/tf-datasets/titanic/eval.csv'

In [0]:
train,test=pd.read_csv(train_url),pd.read_csv(test_url)

In [6]:
print(train.head())

   survived     sex   age  ...     deck  embark_town  alone
0         0    male  22.0  ...  unknown  Southampton      n
1         1  female  38.0  ...        C    Cherbourg      n
2         1  female  26.0  ...  unknown  Southampton      y
3         1  female  35.0  ...        C  Southampton      n
4         0    male  28.0  ...  unknown   Queenstown      y

[5 rows x 10 columns]


In [0]:
l_x=train.pop('survived')
l_y=test.pop('survived')

In [8]:
print(l_x.head())

0    0
1    1
2    1
3    1
4    0
Name: survived, dtype: int64


In [0]:
numeric_col=['age','fare']
categorical_col=['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']

In [0]:
feature_cols=[]

for each in numeric_col:
  feature_cols.append(tf.feature_column.numeric_column(each,dtype=tf.float32))
for each in categorical_col:
  vocab=train[each].unique()
  feature_cols.append(tf.feature_column.categorical_column_with_vocabulary_list(each,vocab))

In [0]:
#Let's build an input pipeline

def pipeline(data,labels,epochs=10,batch=32,shuffle=True):
  def input_line():
    ds=tf.data.Dataset.from_tensor_slices((dict(data),labels))
    if shuffle:
      ds=ds.shuffle(1000)
    ds=ds.batch(batch).repeat(epochs)
    return ds
  return input_line

In [0]:
train=pipeline(train,l_x)
test=pipeline(test,l_y,epochs=1,shuffle=False)

In [15]:
classifier=tf.estimator.LinearClassifier(feature_columns=feature_cols)
classifier.train(train)
result=classifier.evaluate(test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpyvt5305m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa603d35898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constra